## Fitting Synchrotron and IC emission from an electron distribution

In [2]:
#!/usr/bin/env python
import astropy.units as u
import numpy as np
from astropy.io import ascii

import naima
from naima.models import ExponentialCutoffPowerLaw, InverseCompton, Synchrotron

## Read data

# We only consider every fifth X-ray spectral point to speed-up calculations for this example
# DO NOT do this for a final analysis!
soft_xray = ascii.read("RXJ1713_Suzaku-XIS.dat")[::5]
vhe = ascii.read("RXJ1713_HESS_2007.dat")

## Model definition


def ElectronSynIC(pars, data):

    # Match parameters to ECPL properties, and give them the appropriate units
    amplitude = 10 ** pars[0] / u.eV
    alpha = pars[1]
    e_cutoff = (10 ** pars[2]) * u.TeV
    B = pars[3] * u.uG

    # Initialize instances of the particle distribution and radiative models
    ECPL = ExponentialCutoffPowerLaw(amplitude, 10.0 * u.TeV, alpha, e_cutoff)
    # Compute IC on CMB and on a FIR component with values from GALPROP for the
    # position of RXJ1713
    IC = InverseCompton(
        ECPL,
        seed_photon_fields=[
            "CMB",
            ["FIR", 26.5 * u.K, 0.415 * u.eV / u.cm ** 3],
        ],
        Eemin=100 * u.GeV,
    )
    SYN = Synchrotron(ECPL, B=B)

    # compute flux at the energies given in data['energy']
    model = IC.flux(data, distance=1.0 * u.kpc) + SYN.flux(
        data, distance=1.0 * u.kpc
    )

    # The first array returned will be compared to the observed spectrum for
    # fitting. All subsequent objects will be stored in the sampler metadata
    # blobs.
    return model, IC.compute_We(Eemin=1 * u.TeV)


## Prior definition


def lnprior(pars):
    """
    Return probability of parameter values according to prior knowledge.
    Parameter limits should be done here through uniform prior ditributions
    """
    # Limit norm and B to be positive
    logprob = (
        naima.uniform_prior(pars[0], 0.0, np.inf)
        + naima.uniform_prior(pars[1], -1, 5)
        + naima.uniform_prior(pars[3], 0, np.inf)
    )

    return logprob


if __name__ == "__main__":

    ## Set initial parameters and labels
    # Estimate initial magnetic field and get value in uG
    B0 = 2 * naima.estimate_B(soft_xray, vhe).to("uG").value

    p0 = np.array((33, 2.5, np.log10(48.0), B0))
    labels = ["log10(norm)", "index", "log10(cutoff)", "B"]

    ## Run sampler
    sampler, pos = naima.run_sampler(
        data_table=[soft_xray, vhe],
        p0=p0,
        labels=labels,
        model=ElectronSynIC,
        prior=lnprior,
        nwalkers=32,
        nburn=100,
        nrun=20,
        threads=4,
        prefit=True,
        interactive=False,
    )

    ## Save run results to HDF5 file (can be read later with naima.read_run)
    naima.save_run("RXJ1713_SynIC.hdf5", sampler)

    ## Diagnostic plots
    naima.save_diagnostic_plots(
        "RXJ1713_SynIC",
        sampler,
        sed=True,
        blob_labels=["Spectrum", "$W_e$($E_e>1$ TeV)"],
    )
    naima.save_results_table("RXJ1713_SynIC", sampler)

INFO: Finding Maximum Likelihood parameters through Nelder-Mead fitting... [naima.core]
INFO:    Initial parameters: [31.85848977  2.5         1.68124124 18.43093967] [naima.core]
INFO:    Initial lnprob(p0): -1143.008 [naima.core]
INFO:    New ML parameters : [32.13161947  2.60030353  1.72265098 15.26914507] [naima.core]
INFO:    Maximum lnprob(p0): -51.177 [naima.core]
Burning in the 32 walkers with 100 steps...

Progress of the run: 0 percent (0 of 100 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.1             2.6            1.72            15.3      
  Last ensemble std    :      0.161          0.0152          0.00902         0.0752     
  Last ensemble lnprob :  avg: -7594.890, max: -53.289

Progress of the run: 5 percent (5 of 100 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.1             2.6            1.72   


Progress of the run: 5 percent (1 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.1            2.53            1.69             15       
  Last ensemble std    :     0.00565         0.0465           0.018           0.224     
  Last ensemble lnprob :  avg: -51.218, max: -49.609

Progress of the run: 10 percent (2 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.1            2.52            1.69             15       
  Last ensemble std    :     0.00546         0.0466          0.0183           0.225     
  Last ensemble lnprob :  avg: -51.214, max: -49.657

Progress of the run: 15 percent (3 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.1            2.53            1.69             15       
  Last ensemble std    :     0

/home/born-again/anaconda3/envs/gammapy-1.1/lib/python3.9/site-packages/naima/analysis.py:163: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  f.clf()


INFO: Plotting $W_e$($E_e>1$ TeV)... [naima.analysis]
INFO: Saving results table in RXJ1713_SynIC_results.ecsv [naima.analysis]


In [2]:
from astropy.io import ascii

soft_xray = ascii.read("RXJ1713_Suzaku-XIS.dat")[::5]
vhe = ascii.read("RXJ1713_HESS_2007.dat")

In [3]:
soft_xray

energy,flux,flux_error
eV,erg / (s cm2),erg / (s cm2)
float64,float64,float64
549.326,4.53469e-10,2.03377e-11
883.304,4.15769e-10,1.16404e-11
1003.76,4.25126e-10,1.04895e-11
1076.738,3.97387e-10,8.71507e-12
1149.741,3.88445e-10,7.56421e-12
1222.757,4.02945e-10,6.98812e-12
...,...,...
4996.892,2.62909e-10,6.61456e-12


In [4]:
vhe

energy,energy_edge_lo,energy_edge_hi,flux,flux_error,ul
TeV,TeV,TeV,1 / (TeV s cm2),1 / (TeV s cm2),
float64,float64,float64,float64,float64,int64
0.33,0.3,0.37,2.29e-10,3.2e-11,0
0.4,0.37,0.44,1.25e-10,1.6e-11,0
0.49,0.44,0.54,9.46e-11,9e-12,0
0.59,0.54,0.65,6.06e-11,5.2e-12,0
0.71,0.65,0.79,4.37e-11,3.1e-12,0
0.86,0.79,0.95,2.15e-11,1.8e-12,0
...,...,...,...,...,...
22.18,20.24,24.5,9.24e-15,3.16e-15,0
